In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## artist_id 别名规范化字典
alias_map 别名 -> 实名 

In [2]:
alias_map = np.load("obj/alias_map.npy",allow_pickle=True).item()

# 1. 从文件中构造原始矩阵

In [3]:
"""
记录id到矩阵索引的映射
"""
user_id_index = {}
user_cnt = 0 # index 计数
artist_id_index = {}
artist_cnt = 0 # index 计数


# 测试集保留
test = {}  #（ (user,artist) : score )
# 训练集记录
train_map = {}  # ((user_index,artist_index) : score)

# 构造训练集和测试集
import random
def add_item(user, artist, score):
    """
    需要 alias_map
    20%几率将数据置0，作为测试集
    """
    # artist 别名规范化
    artist = alias_map.get(artist,artist)
    
    # 测试集：从训练集中随机抽取20%记录score作为测试，并置score=0写入训练集
    import random
    if random.random() < 0.2:
        test[(user,artist)] = score
        score = 0
        
    # 训练集
    '''
    user in index ? index : new index, new row
    artist in index ? index : new index, new col
    '''
    global user_cnt
    user_index=user_cnt
    if user in user_id_index:
        user_index = user_id_index[user]
    else:
        user_id_index[user] = user_index
        user_cnt +=1
        
    global artist_cnt
    artist_index=artist_cnt
    if artist in artist_id_index:
        artist_index = artist_id_index[artist]
    else:
        artist_id_index[artist] = artist_index
        artist_cnt +=1
    
    return (user_index,artist_index),score


# 读取文件
n=0
eno =0

import random

with open("user_artist_data.txt",encoding="utf8") as f:
    # 文件总计2400+w行
    for l in f:
#         if random.random() < 0.1:
            n=n+1
            m = re.match(r"(\d+) (\d+) (\d+)",l)
            eles = m.groups() if m is not None else () 
            if len(eles) is 3:
                key,value = add_item(eles[0], eles[1], int(eles[2]))
                train_map[key] = value
            else:
                eno+=1
    #             print(str(n)+" - error: "+l)
            if n%100000 is 0:
                print(f"compete {n}", end="\r")

print("error count : "+str(eno))
print("lines count : "+str(n))

error count : 00
lines count : 24296858


In [5]:
index = pd.DataFrame( [ [_[0],_[1]] for _ in train_map.keys()] , columns=['user','item'])

In [52]:
# 听歌很少的人
index["user"].value_counts()[ index["user"].value_counts() <= 2]

44533     2
70495     2
19760     2
10952     2
68734     2
135997    2
20856     2
4345      2
9899      2
35919     2
51536     2
68618     2
118977    2
35916     2
51770     2
64225     2
134410    2
91807     2
59451     2
124999    2
135560    2
60408     2
15131     2
146257    2
10366     2
146107    2
80388     2
53620     2
141422    2
43916     2
         ..
20786     1
6144      1
58309     1
135550    1
140188    1
61226     1
120894    1
66295     1
84496     1
71720     1
95368     1
107493    1
43079     1
113764    1
147809    1
130154    1
4405      1
131738    1
141330    1
71735     1
108652    1
62655     1
123893    1
65174     1
18997     1
296       1
144600    1
20794     1
104490    1
73482     1
Name: user, Length: 5755, dtype: int64

In [55]:
# 小众歌曲
index["item"].value_counts()[ index["item"].value_counts() <= 2]

583885     2
1039980    2
1036953    2
908687     2
920281     2
776477     2
892435     2
960542     2
352049     2
1026107    2
529583     2
1055514    2
699067     2
298015     2
417622     2
859703     2
809106     2
1113207    2
610883     2
1056596    2
1153830    2
774681     2
166732     2
514204     2
1248998    2
822555     2
1103605    2
1265390    2
1236428    2
1105740    2
          ..
320709     1
287957     1
271581     1
369901     1
402589     1
337149     1
713989     1
681237     1
664861     1
779557     1
746805     1
517285     1
418965     1
1337044    1
140317     1
1320668    1
1435364    1
1418988    1
1402612    1
1386236    1
156693     1
222261     1
435341     1
205885     1
58437      1
1212344    1
1195952    1
107629     1
1245096    1
1089012    1
Name: item, Length: 1195685, dtype: int64

In [6]:
unpop_music = set( index["item"].value_counts()[ index["item"].value_counts() <= 2].index )

In [7]:
train_map_pop = {key: value for key, value in train_map.items() if key[1] not in unpop_music}

artist_cnt_pop = len( set( [ _[1] for _ in train_map_pop.keys() ] ) ) 

user_cnt = len( set( [ _[0] for _ in train_map_pop.keys() ] ) ) 

In [9]:
np.save("obj/train_map_pop.npy", train_map_pop)
np.save("obj/user_id_index.npy", user_id_index)
np.save("obj/artist_id_index.npy", artist_id_index)

In [10]:
# 释放内存
del train_map
import gc
gc.collect()

7

In [ ]:
# 读取缓存数据
train_map_pop = np.load("obj/train_map_pop.npy").item()
artist_cnt_pop = 372441
user_cnt = 147887

# 2. 矩阵分解（梯度下降法）

train_map 为原始矩阵 $M$

分解即  $R \approx PQ = \hat{R} $

初始化 $P Q$ 矩阵 分别为 $m*k, k*n$

In [184]:
# 初始化训练参数
step = 10
a = 0.00001
b = 0.0001

In [185]:
# 定义隐含主题个数
K = 5 
# 初始化矩阵
m,n = user_cnt, artist_cnt
p = np.random.rand(m,K)
q = np.random.rand(K,n)

def loger(x):
    if x<=2:
        return 1
    else:
        return np.log2(x)

def astep():
    n = 0
    loss = 0 # 初始化损失项
    reg = 0 # 初始化正则项
    for i,j in list( train_map.keys()):
        if n%200000 == 0 :
            print(f"processing  {n/241333:.2f}%  loss = {loss:.2f}", end="\r")
        n+=1
        value = loger( train_map.get((i,j)) )
        eij = value - np.dot(p[i,:],q[:,j])
        if math.isinf(eij):
            print(i,j,eij,reg,loss,n,p[i,:])
            return loss
        p[i,:] = p[i,:] + a*(eij*q[:,j] - b*p[i,:])
        q[:,j] = q[:,j] + a*(eij*p[i,:] - b*q[:,j])
        reg += np.sum( p[i,:]**2 + q[:,j]**2 )
        loss += ( value - np.dot(p[i,:],q[:,j]) )**2
    print(f"processing  100%  loss = {loss:.2f}", end="\r")
    return loss + (b/2)*reg

In [ ]:
loss = astep()

In [154]:
i, j = 43 ,824
loss = 0
reg = 0
eij = value - np.dot(p[i,:],q[:,j])
value = train_map.get((i,j))
reg += np.sum( p[i,:]**2 + q[:,j]**2 )
loss += ( value - np.dot(p[i,:],q[:,j]) )**2

/Users/chenzhihao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in square
  
/Users/chenzhihao/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys


In [155]:
eij, value, reg, loss

(-1.507628953676668e+180, 69, inf, inf)

In [151]:
p[i,:],q[:,j]

array([57.91136253, 83.68032023, 64.3401105 , 83.27776538, 70.00792062])

In [11]:
try:
    
    
    
    
    
    finally:
    # 保存数据
    
        
except KeyboardInterrupt:
    
        
        

result 1


In [168]:
m,n

(148111, 1568126)